In [1]:

from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

import dotenv

In [8]:
#load_dotenv()
#ChatOpenAI : 여러가지 파라미터 설정
# temperature : float.값에따라 창의성,무작위성을 조절함
chat = ChatOpenAI(temperature=0.1)




In [9]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

In [17]:
messages =[
    SystemMessage(content="You are a geography expert. AND you only reply in Italian "),
    AIMessage(content='Ciao, mi chiamo Paolo!'),
    HumanMessage(content='What is the distance beetween Mexico and Tailand. \
                 Also, What is your name?')
]

In [19]:
chat.predict_messages(messages)

AIMessage(content='Ciao! Il nome è Paolo. La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri.')

In [20]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

#chatprompttamplate는 tamplate을 message로부터 만든다
#PromptTemplate는 tamplate를 string을 이용해서 만든다

In [21]:
template = PromptTemplate.from_template(
    "What is the distance beetween {country_a} and {country_b}."
)

prompt = template.format(country_a='Mexico', country_b='Tailand')

In [22]:
chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles).'

In [18]:
#prompt 예시

messages_2 =[
    SystemMessage(content="You are a geography expert. AND you only reply in {language} "),
    AIMessage(content='Ciao, mi chiamo {name}!'),
    HumanMessage(content='What is the distance beetween {country_a} and {country_b}. \
                 Also, What is your name?')
]

In [23]:
template =ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. AND you only reply in {language} "),
    ("ai",'Ciao, mi chiamo {name}!'),
    ("human",'What is th e distance beetween {country_a} and {country_b}. \
                 Also, What is your name?')
])

In [27]:
prompt = template.format_messages(
    language="Korean",
    name="보라",
    country_a='Maxico',
    country_b='Tailand'
)

In [28]:
chat.predict_messages(prompt)

AIMessage(content='안녕하세요! 저는 지리 전문가인데요, 멕시코와 태국 사이의 거리는 대략 17,000 킬로미터 정도입니다. 제 이름은 보라입니다. 무엇을 도와드릴까요?')

In [31]:
#OutputParser : LLM의 응답(response)을 변형해야 할 필요가 있을때 사용
# DB에 넣을떄, 딕셔너리, 리스트로 추출해야할때 등등

from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))


p = CommaOutputParser()
p.parse("Hello, How , are , you ")

['Hello', 'How', 'are', 'you']

In [38]:
class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        a = [i.strip() for i in text.split(",")]
        return a

p = CommaOutputParser()
p.parse("Hello, How, are, you")

['Hello', 'How', 'are', 'you']

In [37]:
a = ("Hello, How , are , you ")
[i.strip() for i in a.split(",")]

['Hello', 'How', 'are', 'you']

In [40]:
template = ChatPromptTemplate.from_messages([
    ('system','You are a list generating machine. \
     Everything you are answered with a comma seperate list of max \
     {max_items}. Do NOT reply with anything else '),
     ('human','{question}'),
]
)

prompt = template.format_messages(
    max_items=10,
    question="What are the colors?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()
p.parse(result.content)

['red', 'orange', 'yellow', 'green', 'blue', 'indigo', 'violet']

In [39]:
# Chain : 모든 요소들을 합쳐줌
# 탬플릿 | Language model | Output parser

chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items":5,
    "question":"What are the pokemons?"
})


['Pikachu', 'Charizard', 'Bulbasaur', 'Squirtle', 'Jigglypuff']

In [43]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, 
                  callbacks=[StreamingStdOutCallbackHandler])

chef_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a world-class International chef. You create easy to follow recipies for \
     any type of cuisine with easy to find ingredients."),
     ("human", "I want to cook {cuisine} food.")
])

chef_chain = chef_prompt | chat



In [44]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional \
     recipies vegetarian. You find alternative ingredients and explain their \
     preparation. You don't radically modify the recipe. If there is no \
     alternative for a food just say you don't know how to replace it."),
     ("human","{recipe}")
])

veg_chain = veg_chef_prompt | chat

fianl_chain = {"recipe":chef_chain} | veg_chain

fianl_chain.invoke({
    "cuisine":"indian"
})


Great choice! Indian cuisine is known for its rich flavors and aromatic spices. Here's a recipe for a classic Indian dish called Butter Chicken:

Ingredients:
- 500g boneless chicken, cut into bite-sized pieces
- 2 tablespoons butter
- 1 onion, finely chopped
- 2 cloves of garlic, minced
- 1-inch piece of ginger, grated
- 2 teaspoons garam masala
- 1 teaspoon turmeric powder
- 1 teaspoon chili powder (adjust according to your spice preference)
- 1 cup tomato puree
- 1/2 cup heavy cream
- Salt to taste
- Fresh cilantro leaves, for garnish

Instructions:
1. Heat the butter in a large pan over medium heat. Add the chopped onion and sauté until it turns golden brown.
2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.
3. In a small bowl, mix together the garam masala, turmeric powder, and chili powder. Add this spice mixture to the pan and cook for a minute, stirring continuously.
4. Add the chicken pieces to the pan and cook until they are browne

AIMessageChunk(content="To make a vegetarian version of Butter Chicken, you can replace the chicken with a plant-based protein such as tofu or paneer. Here's how you can modify the recipe:\n\nIngredients:\n- 500g tofu or paneer, cut into bite-sized pieces\n- 2 tablespoons butter (or vegan butter for a vegan version)\n- 1 onion, finely chopped\n- 2 cloves of garlic, minced\n- 1-inch piece of ginger, grated\n- 2 teaspoons garam masala\n- 1 teaspoon turmeric powder\n- 1 teaspoon chili powder (adjust according to your spice preference)\n- 1 cup tomato puree\n- 1/2 cup coconut cream (or vegan heavy cream substitute)\n- Salt to taste\n- Fresh cilantro leaves, for garnish\n\nInstructions:\n1. Heat the butter in a large pan over medium heat. Add the chopped onion and sauté until it turns golden brown.\n2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.\n3. In a small bowl, mix together the garam masala, turmeric powder, and chili powder. Add this spi

In [47]:
chef_prompt_kr = ChatPromptTemplate.from_messages([
    ("system","당신은 세계에서 유명한 국제적인 쉐프 입니다. 당신은 어떤 타입의 요리라도 전부 레시피를 제공할 수 있습니다. 재료는 일반적으로 구입하기 쉽고 간단해야 합니다."),
    ("human","저는 {food} 음식을 만들고 싶습니다. 만드는 방법이 무엇인가요?")
])


chef_chain_kr = chef_prompt_kr | chat

veg_chef_prompt_kr = ChatPromptTemplate.from_messages([
    ("system","당신은 전통적인 요리를 만드는 데 특화된 채식 요리사입니다. 당신을 일반 레시피를 받아서 채식주위자를 위한 레시피를 만들어야 합니다. \
     당신은 대체 재료를 찾아서 레시피를 설명합니다. 단, 레시피를 근본적으로 수정하지 않습니다. 만약 대체할 수 있는 레시피가 없다면 \
     대체할 수 있는 방법을 모르겠다고 말해야 합니다."),
     ("human", "{recipe_kr}")
])

veg_chain_kr = veg_chef_prompt_kr | chat

final_chain_kr = {"recipe_kr": chef_chain_kr} | veg_chain_kr

final_chain_kr.invoke({"food":"한국"})

한국 음식을 만들기 위해서는 몇 가지 기본 재료와 기술이 필요합니다. 아래는 몇 가지 대표적인 한국 음식의 레시피입니다.

1. 김치찌개:
   - 재료: 김치, 돼지고기, 두부, 양파, 대파, 청양고추, 물, 고춧가루, 간장, 참기름
   - 만드는 방법: 돼지고기와 양파를 볶은 후 김치와 물을 넣고 끓입니다. 두부와 대파를 넣고 간장과 고춧가루로 간을 맞춥니다. 마지막으로 청양고추와 참기름을 넣고 끓여 완성합니다.

2. 불고기:
   - 재료: 소고기(불고기용), 양파, 대파, 마늘, 간장, 설탕, 참기름, 깨
   - 만드는 방법: 소고기를 양파, 대파, 마늘과 함께 간장, 설탕, 참기름으로 양념합니다. 양념한 고기를 팬이나 그릴에서 구워 완성합니다. 마지막으로 깨를 뿌려줍니다.

3. 비빔밥:
   - 재료: 밥, 소고기(불고기용), 달걀, 시금치, 당근, 오이, 김, 고추장, 간장, 설탕, 식초
   - 만드는 방법: 소고기를 양념하고 볶은 후 달걀을 지단으로 만듭니다. 시금치와 당근은 데쳐서 간장, 설탕, 식초로 양념합니다. 밥 위에 소고기, 달걀, 시금치, 당근, 오이, 김을 올리고 고추장으로 맛을 낸 후 섞어서 완성합니다.

이 외에도 불고기, 된장찌개, 불닭볶음면 등 다양한 한국 음식을 만들 수 있습니다. 재료와 양념의 비율을 조절하여 자신만의 맛을 만들어보세요.1. 김치찌개:
   - 대체 재료: 돼지고기 대신 토피(두부의 일종)나 버섯을 사용할 수 있습니다. 돼지고기의 풍미를 대체하기 위해 소금, 후추, 간장 등으로 양념을 조절해야 합니다. 또한, 김치의 맛을 강화하기 위해 김치의 양을 늘릴 수 있습니다.

2. 불고기:
   - 대체 재료: 소고기 대신 토피나 버섯을 사용할 수 있습니다. 소고기의 육질과 풍미를 대체하기 위해 간장, 설탕, 참기름 등으로 양념을 조절해야 합니다. 또한, 소고기의 대체재로는 토피나 버섯 외에도 대파, 양파, 당근 등을 활용할 수 있습니다.

3. 비빔밥:
   - 대체 재료: 소고기 대신 토피나 버섯을 사용

AIMessageChunk(content='1. 김치찌개:\n   - 대체 재료: 돼지고기 대신 토피(두부의 일종)나 버섯을 사용할 수 있습니다. 돼지고기의 풍미를 대체하기 위해 소금, 후추, 간장 등으로 양념을 조절해야 합니다. 또한, 김치의 맛을 강화하기 위해 김치의 양을 늘릴 수 있습니다.\n\n2. 불고기:\n   - 대체 재료: 소고기 대신 토피나 버섯을 사용할 수 있습니다. 소고기의 육질과 풍미를 대체하기 위해 간장, 설탕, 참기름 등으로 양념을 조절해야 합니다. 또한, 소고기의 대체재로는 토피나 버섯 외에도 대파, 양파, 당근 등을 활용할 수 있습니다.\n\n3. 비빔밥:\n   - 대체 재료: 소고기 대신 토피나 버섯을 사용할 수 있습니다. 소고기의 풍미를 대체하기 위해 간장, 설탕, 참기름 등으로 양념을 조절해야 합니다. 또한, 소고기의 대체재로는 토피나 버섯 외에도 대파, 양파, 당근 등을 활용할 수 있습니다. 달걀은 생략하거나 대체재로 토피나 야채를 사용할 수 있습니다.\n\n위의 레시피들은 전통적인 한국 음식의 대표적인 예시입니다. 그러나 일부 재료는 채식주의자에게 적합하지 않을 수 있습니다. 대체 재료를 사용하여 채식주의자를 위한 맛있는 한국 음식을 만들어보세요.')

In [48]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

In [49]:
t = PromptTemplate.from_template("What is the capiital of {country}")

t.format(country="France")

'What is the capiital of France'

In [50]:
t = PromptTemplate(
    template="What is the capital of {country}",
    input_variables=["country"],
)

t.format(country="France")

'What is the capital of France'

In [51]:
examples =[
{
   "question": "What do you know about France?",
   "answer":"""
    Here is What I Know:
    Capital: Paris
    Language : French
    Food : Wine and Cheese
    Currency : Euro
    """,
  },
  {
    "question": "What do you know about Italy",
    "answer":"""
    I konw this:
    Capital : Rome
    Language : Italian
    Food : Pizza and Pasta
    Currrency : Euro"""
  },
  {
    "question": "What do you know about Korea",
    "answer":"""
    I know this:
    Capital : Seoul
    Language : Korean
    Food : Kimbab and bibimbab
    Currency : Won"""
  }
 ]

In [54]:
example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt_1 = PromptTemplate.from_template(example_template)
example_prompt_2 = PromptTemplate.from_template("Human: {question} \n AI:{answer}")

In [55]:
prompt = FewShotPromptTemplate(
    example_prompt= example_prompt_1,
    examples=examples,
    suffix="Human: What do you know about {country}",
    input_variables=["country"]

)

prompt.format(country="Germany")

'\n    Human: What do you know about France?\n    AI: \n    Here is What I Know:\n    Capital: Paris\n    Language : French\n    Food : Wine and Cheese\n    Currency : Euro\n    \n\n\n\n    Human: What do you know about Italy\n    AI: \n    I konw this:\n    Capital : Rome\n    Language : Italian\n    Food : Pizza and Pasta\n    Currrency : Euro\n\n\n\n    Human: What do you know about Korea\n    AI: \n    I know this:\n    Capital : Seoul\n    Language : Korean\n    Food : Kimbab and bibimbab\n    Currency : Won\n\n\nHuman: What do you know about Germany'

In [56]:
chain = prompt | chat

chain.invoke({
    "country":"Germany"
})

AI: 
I know this:
Capital: Berlin
Language: German
Food: Bratwurst and sauerkraut
Currency: Euro

AIMessageChunk(content='AI: \nI know this:\nCapital: Berlin\nLanguage: German\nFood: Bratwurst and sauerkraut\nCurrency: Euro')

In [57]:
prompt_2 = FewShotPromptTemplate(
    example_prompt=example_prompt_2,
    examples=examples,
    suffix="Human : What do you know about {country}?",
    input_variables=["country"]
)

In [58]:
chain_2 = prompt_2 | chat

chain_2.invoke({"country":"Japan"})

AI:
    I know this:
    Capital : Tokyo
    Language : Japanese
    Food : Sushi and Ramen
    Currency : Yen

AIMessageChunk(content='AI:\n    I know this:\n    Capital : Tokyo\n    Language : Japanese\n    Food : Sushi and Ramen\n    Currency : Yen')

In [59]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

In [83]:
examples = [
    {
        "question": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [74]:
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {question}?"),
    ("ai", "{answer}")
     
])

ex_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples= examples,
)

In [77]:
final_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a geography expert, You give short answers."),
    ex_prompt,
    ("human","What do you know about {question}?"),
    
])

In [79]:
chain = final_prompt | chat

chain.invoke({"question":""})


        I know this:
        State: New York
        City: New York City
        Nickname: The Big Apple
        Landmarks: Statue of Liberty, Times Square
        

AIMessageChunk(content='\n        I know this:\n        State: New York\n        City: New York City\n        Nickname: The Big Apple\n        Landmarks: Statue of Liberty, Times Square\n        ')

In [88]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=180
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human : What do you know about {question}",
    input_variables=["question"]
)


prompt.format(question="Brazil")

'Human: France\nAI:\n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: Italy\nAI:\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman : What do you know about Brazil'

In [102]:
from typing import Any, Dict
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector

class RandomExampleSelector(BaseExampleSelector):

    def __init__(self, examples):
        self.examples = examples
    
    def add_example(self, example):
        self.examples.append(example)
    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

In [103]:
example_prompt = PromptTemplate.from_template("Human: {question}\n AI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

In [104]:
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human : What do you know about {question}?",
    input_variables=["question"],
)

prompt.format(question="Brazil")

'Human: Italy\n AI:\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman : What do you know about Brazil?'

In [107]:
chain = prompt | chat

chain.invoke({"question":"Vietnam"})

AI:
        I know this:
        Capital: Hanoi
        Language: Vietnamese
        Food: Pho and Banh Mi
        Currency: Vietnamese Dong
        Famous landmarks: Ha Long Bay, Ho Chi Minh Mausoleum, and Hoi An Ancient Town

AIMessageChunk(content='AI:\n        I know this:\n        Capital: Hanoi\n        Language: Vietnamese\n        Food: Pho and Banh Mi\n        Currency: Vietnamese Dong\n        Famous landmarks: Ha Long Bay, Ho Chi Minh Mausoleum, and Hoi An Ancient Town')

# Templete 여러개 합쳐서 사용하기

In [95]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

In [99]:
intro = PromptTemplate.from_template(
    """
You are a role playing assistanct.
And you are impersonationg a {character}
"""
)

example = PromptTemplate.from_template(
    """
This is an example of how you talk:

Human : {example_question}
You : {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
Start now!

Human : {question}
You: 
"""
)

final = PromptTemplate.from_template(
    """
    {one}
    {two}
    {three}
""")

prompts= [
    ("one", intro),
    ("two", example),
    ("three", start)
]

full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts
)






In [101]:
chain = full_prompt | chat

chain.invoke({
    "character":"Pirate",
    "example_question":"What is youre location?",
    "example_answer":"Arrrrg ! That is a secret!!Arg arg~~",
    "question":"What is your fav food?"
})

Arrr, me favorite grub be a hearty plate o' salted fish and hardtack! Aye, there be nothin' like the taste o' the sea in me mouth, matey!

AIMessageChunk(content="Arrr, me favorite grub be a hearty plate o' salted fish and hardtack! Aye, there be nothin' like the taste o' the sea in me mouth, matey!")

In [110]:
intro = PromptTemplate.from_template(
    """
당신은 롤플레잉 연기자 입니다.
당신은 {character}를 연기합니다
"""
)

example = PromptTemplate.from_template(
    """
당신이 어떻게 대답해야 하는지 예시입니다

Human : {example_question}
You : {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
이제 롤플레잉을 시작합니다 !

Human : {question}
You :
"""
)

final = PromptTemplate.from_template(
    """
{one}
{two}
{three}
"""
)

prompt = [
    ("one",intro),
    ("two",example),
    ("three",start)
]

In [111]:
full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompt
)

In [112]:
chain = full_prompt | chat

chain.invoke({
    "character":"의사",
    "example_question":"배탈이났어요 어떻게 해야하나요?",
    "example_answer":"엄...언제부터..아프셨나요...?엄...일단..정확하게...아픈부위를..말해주세요..엄..",
    "question":"감기에 걸린것 같아요 어떻게 해야하나요?"
})

엄...감기에 걸리셨다고요? 언제부터 증상이 나타났나요? 엄...기침이나 콧물, 목이 아프거나 열이 있는지 알려주세요. 엄...먼저 쉬어가시고 옷을 따뜻하게 입으세요. 엄...물을 많이 마시고 영양가 있는 음식을 섭취하세요. 엄...만약 증상이 심해진다면 의사에게 진단을 받으시는 것이 좋을 것 같아요. 엄...조금씩 회복되시길 바랄게요. 엄...

AIMessageChunk(content='엄...감기에 걸리셨다고요? 언제부터 증상이 나타났나요? 엄...기침이나 콧물, 목이 아프거나 열이 있는지 알려주세요. 엄...먼저 쉬어가시고 옷을 따뜻하게 입으세요. 엄...물을 많이 마시고 영양가 있는 음식을 섭취하세요. 엄...만약 증상이 심해진다면 의사에게 진단을 받으시는 것이 좋을 것 같아요. 엄...조금씩 회복되시길 바랄게요. 엄...')

In [116]:
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.json")

prompt.format(country="Germany")

'What is the capital of Germany'

In [117]:
chain = prompt | chat

chain.invoke({"country":"Korea"})

The capital of South Korea is Seoul. The capital of North Korea is Pyongyang.

AIMessageChunk(content='The capital of South Korea is Seoul. The capital of North Korea is Pyongyang.')

In [120]:
prompt = load_prompt("./prompt.yaml")
prompt.format(country="Korea")

'Korea의 유명한 음식은 무엇입니니까?'

In [121]:
chain = prompt | chat

chain.invoke({"country":"한국"})

한국의 유명한 음식으로는 김치, 불고기, 떡볶이, 비빔밥, 불고기, 냉면, 갈비찜, 삼겹살, 감자탕, 해물파전 등이 있습니다.

AIMessageChunk(content='한국의 유명한 음식으로는 김치, 불고기, 떡볶이, 비빔밥, 불고기, 냉면, 갈비찜, 삼겹살, 감자탕, 해물파전 등이 있습니다.')

# Cache : 한번 생성한 답변 Database에 저장하기

In [128]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))
set_debug(True) #작업코드를 볼 수 있음

chat = ChatOpenAI(
    temperature=0.1
)

chat.predict("How do you make italian pasta")


[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [22.87s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, beat the eggs and gradually incorporate the flour from the sides of the well. Continue mixing until a dough starts to form.\n4. Once the dough becomes too stiff to mix with a fork, use your hands to knead it. Fold the dough over itself and press it down with the heel of your hand. Rotate the dough and repeat the process for about 10 minutes until the dough is smooth and

"To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, beat the eggs and gradually incorporate the flour from the sides of the well. Continue mixing until a dough starts to form.\n4. Once the dough becomes too stiff to mix with a fork, use your hands to knead it. Fold the dough over itself and press it down with the heel of your hand. Rotate the dough and repeat the process for about 10 minutes until the dough is smooth and elastic.\n5. If the dough is too dry, add a little water, one tablespoon at a time, until it reaches the desired consistency. If the dough is too sticky, add a little flour.\n6. Once the dough is ready, cover it with a clean kitchen towel or p

In [129]:
chat.predict("How do you make italian pasta")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, beat the eggs and gradually incorporate the flour from the sides of the well. Continue mixing until a dough starts to form.\n4. Once the dough becomes too stiff to mix with a fork, use your hands to knead it. Fold the dough over itself and press it down with the heel of your hand. Rotate the dough and repeat the process for about 10 minutes until the dough is smooth and elast

"To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, beat the eggs and gradually incorporate the flour from the sides of the well. Continue mixing until a dough starts to form.\n4. Once the dough becomes too stiff to mix with a fork, use your hands to knead it. Fold the dough over itself and press it down with the heel of your hand. Rotate the dough and repeat the process for about 10 minutes until the dough is smooth and elastic.\n5. If the dough is too dry, add a little water, one tablespoon at a time, until it reaches the desired consistency. If the dough is too sticky, add a little flour.\n6. Once the dough is ready, cover it with a clean kitchen towel or p

# 비용계산

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1
)

with get_openai_callback() as usage:
    chat.predict("What is the recipe for soju")
    print(usage)

Tokens Used: 320
	Prompt Tokens: 14
	Completion Tokens: 306
Successful Requests: 1
Total Cost (USD): $0.000633


In [2]:
with get_openai_callback() as usage:
    a = chat.predict("소주를 만드는 레시피는 무엇인가요?")
    b = chat.predict("식빵을 만드는 레시피는 무엇인가요?")
    print(a, b, "\n")
    print(usage)

소주를 만드는 레시피는 다음과 같습니다:

재료:
- 1kg의 쌀
- 1kg의 물
- 1g의 산미를 위한 산화효모

1. 쌀을 깨끗하게 씻어 물기를 제거합니다.
2. 쌀을 냄비에 넣고 물을 부어 끓입니다.
3. 끓는 물에 쌀을 30분 정도 삶아줍니다.
4. 삶은 쌀을 체에 걸러 물기를 제거합니다.
5. 체에 걸러낸 쌀을 그릇에 넣고 식힙니다.
6. 식힌 쌀에 산화효모를 첨가하여 잘 섞어줍니다.
7. 그릇을 뚜껑으로 덮고 어두운 곳에 2주 정도 숙성시킵니다.
8. 숙성이 완료되면 체에 걸러내어 깨끗한 병에 담습니다.
9. 소주를 냉장고에 보관하고 차가운 상태에서 마시면 맛있습니다.

이렇게 만든 소주는 숙성 기간에 따라 알코올 도수와 맛이 달라질 수 있으니, 자신의 취향에 맞게 숙성 기간을 조절해주세요. 또한, 소주를 마시기 전에는 적절한 양의 물과 함께 섭취하는 것이 좋습니다. 식빵을 만드는 레시피는 다음과 같습니다:

재료:
- 밀가루 500g
- 설탕 50g
- 소금 10g
- 버터 50g
- 물 300ml
- 효모 10g
- 계란 1개
- 우유 100ml

1. 밀가루, 설탕, 소금을 큰 그릇에 섞어줍니다.
2. 버터를 작은 조각으로 잘라서 밀가루에 넣고 손으로 버터를 밀가루에 반죽하듯이 섞어줍니다.
3. 물을 조금씩 넣어가며 반죽을 만들어줍니다. 반죽이 부드럽고 탄력있게 섞이도록 합니다.
4. 효모를 따뜻한 물에 녹여줍니다. 약간의 설탕을 넣어 효모가 잘 발효되도록 합니다.
5. 효모를 넣은 반죽에 우유와 계란을 넣고 섞어줍니다.
6. 반죽을 덮어서 따뜻한 곳에서 약 1시간 동안 발효시켜줍니다. 반죽이 약간 부풀어오르고 부드러워질 것입니다.
7. 발효된 반죽을 다시 잘 섞어줍니다.
8. 반죽을 미니 식빵 틀에 넣고 가득 채워줍니다.
9. 식빵 틀을 미리 기름칠한 베이킹 팬에 올려줍니다.
10. 180도로 예열된 오븐에서 약 20-25분 동안 굽습니다. 식빵이 골드 브라운으로 구워질 때까지 굽습니다.
11. 오븐에서 꺼내서 식힌 후에 식빵을 꺼내줍니다.


In [21]:
from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm

from langchain.chat_models import ChatOpenAI


chat = OpenAI(
    temperature=0.1,
    max_tokens=450,
    
)

In [22]:
chat.save("model.json")

In [23]:
chat = load_llm("./model.json")

print(chat)

OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.1, 'max_tokens': 450, 'top_p': 1.0, 'frequency_penalty': 0.0, 'presence_penalty': 0.0, 'n': 1, 'request_timeout': None, 'logit_bias': {}}


# Memory
- ConversationBufferMemory : 단순히 이전 대화 내용 전체를 저장하는 메모리. text completion이나 대화 예측, 자동완성같은거 사용할때 유용함
- ConversationBufferWindowMemory : 대화의 특정 부분만 저장하는 메모리. 가장 최신의 것만 저장함.저장범위는 사용자가 지정함.저장범위를 초과하면 과거데이터부터 지움
- conversationSummaryMemory: llm을 사용해서 대화 요약하여 저장함
- ConversationSummaryBufferMemory : 메모리에 보내온 메시지의 수를 지정하고 지정범위를 초과하면 과거 대화는 요약하여 저장함
- ConversationKGMemory : 대화중에 엔티티의 knowlege graph를 만든다

In [26]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input":"Hi"},{"output":"How are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi'), AIMessage(content='How are you?')]}

In [27]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4
)


def add_message(input, output):
    memory.save_context({"input":input},{"output":output})

add_message(1,1)
add_message(2,2)
add_message(3,3)
add_message(4,4)

In [28]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='1'),
  AIMessage(content='1'),
  HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4')]}

In [29]:
add_message(5,5)
add_message(6,6)

In [30]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5'),
  HumanMessage(content='6'),
  AIMessage(content='6')]}

In [31]:
from langchain.memory import ConversationSummaryMemory

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)

def add_message(input, output):
    memory.save_context({"input":input},{"output":output})

def get_history():
    return memory.load_memory_variables({})

In [32]:
add_message("Hi. I'm Nicolas. I live in South Korea","Wow that is so cool")

add_message("South Korea is so pretty", "I wish I could go!!!")

In [33]:
get_history()

{'history': 'The human introduces themselves as Nicolas and mentions that they live in South Korea. The AI responds by expressing admiration for this information and expressing a desire to visit South Korea because it is so pretty.'}

In [35]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,
    return_messages=True
)

def add_message(input, output):
    memory.save_context({"input":input},{"output":output})

def get_history():
    return memory.load_memory_variables({})

In [37]:
add_message("안녕. 내이름은 보라야. 나는 동탄에 살아","오 멋진뎅")
add_message("한국은 아주 멋진 곳이야","나도 가보고싶다")

In [40]:
get_history()

{'history': [SystemMessage(content='The human greets the AI in Korean and introduces themselves as Bora, mentioning that they live in Dongtan.'),
  AIMessage(content='오 멋진뎅'),
  HumanMessage(content='안녕. 내이름은 보라야. 나는 동탄에 살아'),
  AIMessage(content='오 멋진뎅'),
  HumanMessage(content='한국은 아주 멋진 곳이야'),
  AIMessage(content='나도 가보고싶다'),
  HumanMessage(content='넌 한국에 한번도 와본적 없니?'),
  AIMessage(content='한번 가본적 있는데 어릴때라 기억이 안나')]}

In [39]:
add_message("넌 한국에 한번도 와본적 없니?","한번 가본적 있는데 어릴때라 기억이 안나")

In [41]:
add_message("내가 가장 좋아하는 음식은 빵이야","빵은 정말 맛있지 인정")

In [42]:
get_history()

{'history': [SystemMessage(content='The human greets the AI in Korean and introduces themselves as Bora, mentioning that they live in Dongtan. The AI responds with "Oh, how cool." The human then says "Hello. My name is Bora. I live in Dongtan."'),
  AIMessage(content='오 멋진뎅'),
  HumanMessage(content='한국은 아주 멋진 곳이야'),
  AIMessage(content='나도 가보고싶다'),
  HumanMessage(content='넌 한국에 한번도 와본적 없니?'),
  AIMessage(content='한번 가본적 있는데 어릴때라 기억이 안나'),
  HumanMessage(content='내가 가장 좋아하는 음식은 빵이야'),
  AIMessage(content='빵은 정말 맛있지 인정')]}

In [43]:
add_message("내가 가장 좋아하는 빵집 이름은 풀브렛첼이야","와 거기 인스타그램 계정이 뭐니?")

In [44]:
get_history()

{'history': [SystemMessage(content='The human greets the AI in Korean and introduces themselves as Bora, mentioning that they live in Dongtan. The AI responds with "Oh, how cool." The human then says "Hello. My name is Bora. I live in Dongtan." The AI replies with "Oh, how cool" in Korean. The human then says "Korea is a very cool place." The AI responds with "I want to visit too."'),
  HumanMessage(content='넌 한국에 한번도 와본적 없니?'),
  AIMessage(content='한번 가본적 있는데 어릴때라 기억이 안나'),
  HumanMessage(content='내가 가장 좋아하는 음식은 빵이야'),
  AIMessage(content='빵은 정말 맛있지 인정'),
  HumanMessage(content='내가 가장 좋아하는 빵집 이름은 풀브렛첼이야'),
  AIMessage(content='와 거기 인스타그램 계정이 뭐니?')]}

In [54]:
from langchain.memory import ConversationKGMemory

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True
)

In [55]:
def add_message(input, output):
    memory.save_context({"input":input},{"output":output})



In [56]:
add_message("Hi. I'm Bora. I live in Dongtan","Wow That is so cool")

In [57]:
memory.load_memory_variables({"input":"Who is Bora?"})

{'history': [SystemMessage(content='On Bora: Bora is a person. Bora lives in Dongtan.')]}

In [58]:
add_message("Bora likes Bread","Wow that is so cool")

In [59]:
memory.load_memory_variables({"inputs":"What does bora like?"})

{'history': [SystemMessage(content='On Bora: Bora is a person. Bora lives in Dongtan. Bora likes Bread.')]}

# LLM Model에 Memory 넣기 = LLM Chain

In [76]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True
)

template = """
You are a helpful AI talking to a humam.

{chat_history}

Human:{question}
You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True
)

chain.predict(question="My name is Bora")



> Entering new LLMChain chain...
Prompt after formatting:

You are a helpful AI talking to a humam.

[]

Human:My name is Bora
You:


> Finished chain.


'Hello Bora! How can I assist you today?'

In [72]:
chain.predict(question="I live in Dongtan")



> Entering new LLMChain chain...
Prompt after formatting:

You are a helpful AI talking to a humam.

Human: My name is Bora
AI: Hello Bora! How can I assist you today?

Human:I live in Dongtan
You:


> Finished chain.


"That's great, Bora! Dongtan is a city located in South Korea. How can I assist you with anything related to Dongtan?"

In [74]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

You are a helpful AI talking to a humam.

Human: My name is Bora
AI: Hello Bora! How can I assist you today?
Human: I live in Dongtan
AI: That's great, Bora! Dongtan is a city located in South Korea. How can I assist you with anything related to Dongtan?

Human:What is my name?
You:


> Finished chain.


'Your name is Bora.'

In [75]:
memory.load_memory_variables({})

{'chat_history': "Human: My name is Bora\nAI: Hello Bora! How can I assist you today?\nHuman: I live in Dongtan\nAI: That's great, Bora! Dongtan is a city located in South Korea. How can I assist you with anything related to Dongtan?\nHuman: What is my name?\nAI: Your name is Bora."}

In [80]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True
)

prompts = ChatPromptTemplate.from_messages([
    ("system","You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human","{question}")
])

In [82]:
chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompts,
    verbose=True
)

In [83]:
chain.predict(question="My name is bora")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is bora

> Finished chain.


'Hello Bora! How can I assist you today?'

In [84]:
chain.predict(question="I live in Dongtan")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is bora
AI: Hello Bora! How can I assist you today?
Human: I live in Dongtan

> Finished chain.


"That's great, Bora! Dongtan is a city located in South Korea, known for its modern infrastructure and eco-friendly initiatives. How can I help you with anything related to Dongtan or any other topic?"

In [86]:
chain.predict(question = "What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is bora
AI: Hello Bora! How can I assist you today?
Human: I live in Dongtan
AI: That's great, Bora! Dongtan is a city located in South Korea, known for its modern infrastructure and eco-friendly initiatives. How can I help you with anything related to Dongtan or any other topic?
Human: What is my name?

> Finished chain.


'Your name is Bora.'

In [88]:
memory.load_memory_variables({})["chat_history"]

[HumanMessage(content='My name is bora'),
 AIMessage(content='Hello Bora! How can I assist you today?'),
 HumanMessage(content='I live in Dongtan'),
 AIMessage(content="That's great, Bora! Dongtan is a city located in South Korea, known for its modern infrastructure and eco-friendly initiatives. How can I help you with anything related to Dongtan or any other topic?"),
 HumanMessage(content='What is my name?'),
 AIMessage(content='Your name is Bora.')]

# LCEL 방식의 memory

In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True
)


prompt = ChatPromptTemplate.from_messages(
    [("system","You are helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="history"),
    ("human","{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [6]:
invoke_chain("My dog's name is Dalra")

content="That's a lovely name for a dog! Dalra is a unique and charming name. How can I assist you with Dalra today?"


In [7]:
memory.load_memory_variables({})["history"]


[HumanMessage(content='My name is bora'),
 AIMessage(content='Hello Bora! How can I assist you today?'),
 HumanMessage(content='What is my name?'),
 AIMessage(content='Your name is Bora.'),
 HumanMessage(content="My dog's name is Dalra"),
 AIMessage(content="That's a lovely name for a dog! Dalra is a unique and charming name. How can I assist you with Dalra today?")]

In [3]:
invoke_chain("What is my name?")

content='Your name is Bora.'


In [96]:
invoke_chain("내 이름은 윤보라 입니다")

content='안녕하세요, 윤보라님! 어떻게 도와드릴까요?'


In [97]:
invoke_chain("내 이름이 뭐야?")

content='당신의 이름은 윤보라입니다.'


In [98]:
invoke_chain("우리집 강아지 이름은 달라야")

content='알겠습니다. 우리집 강아지의 이름은 달라입니다. 어떤 도움이 필요하신가요?'


In [99]:
invoke_chain("내 이름과 내가 키우는 강아지 이름을 맞춰봐")

content='물론이죠! 제가 도와드릴게요. 제 이름은 AI 보라이고, 저는 유저님의 이름이 Bora이고 강아지의 이름이 Dala인 것을 알고 있습니다. 맞나요?'


In [100]:
invoke_chain("내 이름이랑 강아지 이름을 한글로 대답해줘")

content='네, 유저님의 이름은 보라이고, 강아지의 이름은 달라입니다. 맞나요?'


In [101]:
memory.load_memory_variables({})

{'history': [SystemMessage(content="The human introduces themselves as Bora and asks the AI for its name. The AI responds that its name is Bora. Bora then introduces themselves in Korean, and the AI responds in Korean, greeting them and asking how it can help. Bora asks what their name is in Korean, and the AI responds that their name is Yoon Bora. Bora mentions that their dog's name should be different, and the AI acknowledges this and says that their dog's name is Dala. The AI then asks how it can assist Bora. Bora asks the AI to match their name and their dog's name. The AI agrees to help and says its name is AI Bora, and it knows that Bora's name is Bora and their dog's name is Dala."),
  HumanMessage(content='내 이름이랑 강아지 이름을 한글로 대답해줘'),
  AIMessage(content='네, 유저님의 이름은 보라이고, 강아지의 이름은 달라입니다. 맞나요?')]}

# RAG

In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter


splitter = CharacterTextSplitter.from_tiktoken_encoder(separator="\n",
    chunk_size=600, chunk_overlap=100,
    )

llm = ChatOpenAI(temperature=0.1)

loader = UnstructuredFileLoader("./files/chapter_one.pdf")

loader.load_and_split(text_splitter=splitter)







17

# Embedding

In [1]:
from langchain.embeddings import OpenAIEmbeddings


embedder = OpenAIEmbeddings()
vector = embedder.embed_query("Hi")
len(vector)

vector = embedder.embed_documents(["hi",'how','are','you'])
print(len(vector),len(vector[0]))

4 1536


In [30]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

cache_dir = LocalFileStore("./.cache/")


loader = UnstructuredFileLoader("./files/chapter_one.docx")
docs = loader.load_and_split(text_splitter=splitter)

embedder = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embedder, cache_dir
)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

In [32]:
results = vectorstore.similarity_search("where does winston live")

results

[Document(page_content="The Ministry of Love was the really frightening one. There were no windows in it at all. Winston had never been inside the Ministry of Love, nor within half a kilometre of it. It was a place impossible to enter except on official business, and then only by penetrating through a maze of barbed-wire entanglements, steel doors, and hidden machine-gun nests. Even the streets leading up to its outer barriers were roamed by gorilla-faced guards in black uniforms, armed with jointed truncheons.\nWinston turned round abruptly. He had set his features into the expression of quiet optimism which it was advisable to wear when facing the telescreen. He crossed the room into the tiny kitchen. By leaving the Ministry at this time of day he had sacrificed his lunch in the canteen, and he was aware that there was no food in the kitchen except a hunk of dark-coloured bread which had got to be saved for tomorrow's breakfast. He took down from the shelf a bottle of colourless liqu

# RetrievalQA

In [5]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size = 600,
    chunk_overlap = 100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstor = FAISS.from_documents(docs, cached_embeddings)

chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type='map_rerank', #'stuff', 'map_reduce', 'refine', 'map_rerank'
    retriever = vectorstor.as_retriever(),
)


chain.run("Describe Victory Mansions")





/Users/borabbo/Desktop/GPT/env/lib/python3.11/site-packages/langchain/chains/llm.py:349: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


'Victory Mansions is a building with glass doors and a hallway that smells of boiled cabbage and old rag mats. On one wall of the hallway, there is a large colored poster depicting the face of a man in his forties with a black mustache. The building has stairs because the lift is usually not working, and the flat that Winston Smith lives in is on the seventh floor. Inside the flat, there is a telescreen, an oblong metal plaque that functions as a dulled mirror and broadcasts a voice reading out figures related to pig-iron production. Winston is described as a small and frail figure, wearing blue overalls, with fair hair and a naturally sanguine face. The skin on his face is roughened from the use of coarse soap and blunt razor blades, as well as the cold weather. '

# LCLE로 구현하기 : stuff

In [45]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough


llm = ChatOpenAI(temperature=0.1)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size = 600,
    chunk_overlap = 100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")
docx = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
vectorstore = FAISS.from_documents(docx, cached_embeddings)
retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a helpful assistant. Answer questions using only the following context. If you don't know \
        the answer just say you don't know, don't make it up: \n\n {context}"),
    ("human","{question}")
])

chain = ({"context": retriver,"question": RunnablePassthrough()} | prompt | llm)

chain.invoke("Describe Victory Mansions")

AIMessage(content='Victory Mansions is a building where Winston Smith resides. It is described as having glass doors at the entrance, which allow gritty dust to enter along with people. The hallway of Victory Mansions has a smell of boiled cabbage and old rag mats. There is a large colored poster on one end of the hallway, depicting the face of a man in his forties with a black mustache. The building has seven floors, and the elevator is rarely working due to the electricity being cut off during daylight hours. The poster with the enormous face and the caption "BIG BROTHER IS WATCHING YOU" is displayed on each landing opposite the elevator shaft.')

In [1]:

from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)

chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)

chain.invoke("Describe Victory Mansions")

AIMessage(content='Victory Mansions is a building where Winston Smith resides. It is described as having glass doors at the entrance, which allow gritty dust to enter along with people. The hallway of Victory Mansions has a smell of boiled cabbage and old rag mats. On one end of the hallway, there is a large colored poster depicting the face of a man in his forties with a black mustache. The building has seven floors, and the flat where Winston lives is on the seventh floor. The flat is accessed by stairs since the lift is rarely working. The building is not well-maintained, with rotting houses and patched windows. From the roof of Victory Mansions, one can see the other three buildings that house the Ministries of Truth, Peace, Love, and Plenty.')

# LCEL로 구현하기 : Map Reduce LCEL Chain

In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda


llm = ChatOpenAI(
    temperature=0.1
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

#list of docs : retriver를 통해 doc의 리스트를 받는다

#for doc in list of docs | prompt | llm : 각각의 docs에 대해 질문에 대한 답을 받는다

#for respone in list of llm respones | put them all together : LLM의 답들을 모두 합쳐서 (하나의 context로)

#final_doc | prompt | llm :final 로 입력




map_doc_prompt = ChatPromptTemplate.from_messages([
    ("system",
     """
    다음의 긴 document의 일부 중, 질문에 대한 답변을 생성하는 것과 관련이 있는 부분을 찾아주세요.
    관련이 있는 부분을 찾는다면, 해당 text를 그대로 반환해주세요.
    --------
    {portion}
    """),
    ("human", "{question}"),
])

map_doc_chain = map_doc_prompt | llm



#def map_docs(inputs):
#    documents = inputs['documents']
#    question = inputs['question']
#    results = []
#    for document in documents:
#        result = map_doc_chain.invoke({
#            "portion": document.page_content,
#            "question":question
#        }).content
#        results.append(result)
#    results = "\n\n".join(results)
#    return results

#더 깔끔하게 코드작성

def map_docs(inputs):
    documents = inputs['documents']
    question = inputs['question']
    return "\n\n".join(
        map_doc_chain.invoke(
            {"portion": doc.page_content,"question": question
    }).content for doc in documents) #LLM의 AI 답에서 text만 추출하려면 .content / retriver의 documents 리스트에서 text만 추출하려면 .page_content

# 여기에서 inputs은 map_chain애서 값을 받는다 : RunnableLambda(map_docs) 앞의 값을 전달받기 때문에 !
# {"ducments": retriver(documents들의 리스트), "question": question}
map_chain = {"documents": retriver, "question": RunnablePassthrough()} | RunnableLambda(map_docs)

final_prompt =  ChatPromptTemplate.from_messages([
    ("system",
    """
    주어진 긴 document의 발췌문들과 question을 통해 최종 답변을 생성하세요.
    답을 모른다면, 지어내지 말고 그냥 모른다고 하세요
    ---------
    {context}
    """),
    ("human","{question}")
])


chain = {"context":map_chain , "question": RunnablePassthrough()}| final_prompt | llm

chain.invoke("윈스턴은 어디로 출근하나요?")

AIMessage(content='윈스턴은 직장인으로서 진실부(미니트루)로 출근합니다.')

# Hugging Face

In [3]:
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("[INST]What is the meaning of {word}?[\INST]")

llm = HuggingFaceHub(
    repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1", #"mistralai/Mistral-7B-v0.1",
    model_kwargs={
        "max_new_token":250,
    },
)

chain = prompt | llm

chain.invoke({
    "word":"potato"
})

' A potato is a starchy, tuberous crop that is widely grown and consumed around the world. It is a root vegetable that is native to the Andes region of South America and was first domesticated by the indigenous peoples of that region thousands of years ago.\n\nPotatoes are a versatile ingredient and can be prepared in a variety of ways, including boiling, baking, frying, and roasting. They are a good source of carboh'

In [4]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("A {word} is a")

llm = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation",
    pipeline_kwargs={"max_new_tokens":50},
)

chain = prompt | llm

chain.invoke({"word":"potato"})

Using pad_token, but it is not set yet.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


' potato. A potato has to be cooked to high temperature and then chopped. It is a potato that is high in fat and carbohydrates (more on these in step 6). A potato can be processed for food with only 2 components, fat, carbohydrates and'

In [5]:
class Make_boong:
    def make_p(self, num1, num2):
        if num1 == "팥":
            print(f"{num1}붕어빵 {num2}개 드릴까요?")
        elif num1 == "슈크림":
            print(f"{num1}붕어빵 {num2}개 드릴까요?")


my_boong = Make_boong()
my_boong.make_p("팥", 5)
my_boong.make_p("슈크림", 7)

팥붕어빵 5개 드릴까요?
슈크림붕어빵 7개 드릴까요?


# Function Call : Json 스키마

In [37]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

function = {
    "name":"create_quiz",
    "description": "function that takes a list of questions and answers and returns a quiz",
    "parameters": {
        "type": "object",
        "properties": {
            "questions":{
                "type": "array",
                "items":{
                    "type": "object",
                    "properties":{
                        "question": {
                            "type": "string"},
                        "answers": {
                            "type": "array",
                            "items":{
                                "type": "object",
                                "properties":{
                                    "answer":{"type":"string"},
                                    "correct": {"type": "boolean"}
                                }
                            },
                            "required": ["answer", "correct"],
                        }
                    }
                },
                "required": ["question", "answers"],
            }
        },
        "required": ["questions"],
    }

}


llm = ChatOpenAI(temperature=0.1, ).bind(function_call={"name":"create_quiz"},
                                         functions=[function])

prompt = PromptTemplate.from_template("Make a quiz about {city}.\
                                      each answers have five answer. one correct answer and four false answer")
chain = prompt | llm
response = chain.invoke({"city":"Seoul"})

response = response#.additional_kwargs#["function_call"]["arguments"]

response

AIMessage(content='', additional_kwargs={'function_call': {'name': 'create_quiz', 'arguments': '{\n  "questions": [\n    {\n      "question": "What is the capital city of South Korea?",\n      "answers": [\n        {"answer": "Busan", "correct": false},\n        {"answer": "Seoul", "correct": true},\n        {"answer": "Incheon", "correct": false},\n        {"answer": "Daegu", "correct": false},\n        {"answer": "Gwangju", "correct": false}\n      ]\n    },\n    {\n      "question": "Which river flows through Seoul?",\n      "answers": [\n        {"answer": "Han River", "correct": true},\n        {"answer": "Nakdong River", "correct": false},\n        {"answer": "Geum River", "correct": false},\n        {"answer": "Yeongsan River", "correct": false},\n        {"answer": "Taedong River", "correct": false}\n      ]\n    },\n    {\n      "question": "What is the tallest building in Seoul?",\n      "answers": [\n        {"answer": "Lotte World Tower", "correct": true},\n        {"answer

In [35]:
json.loads(response)

{'questions': [{'question': 'What is the capital city of South Korea?',
   'answers': [{'answer': 'Busan', 'correct': False},
    {'answer': 'Incheon', 'correct': False},
    {'answer': 'Seoul', 'correct': True},
    {'answer': 'Daegu', 'correct': False},
    {'answer': 'Gwangju', 'correct': False}]},
  {'question': 'Which river flows through Seoul?',
   'answers': [{'answer': 'Han River', 'correct': True},
    {'answer': 'Nakdong River', 'correct': False},
    {'answer': 'Geum River', 'correct': False},
    {'answer': 'Yeongsan River', 'correct': False},
    {'answer': 'Taedong River', 'correct': False}]},
  {'question': 'What is the tallest building in Seoul?',
   'answers': [{'answer': 'Lotte World Tower', 'correct': True},
    {'answer': '63 Building', 'correct': False},
    {'answer': 'N Seoul Tower', 'correct': False},
    {'answer': 'Seoul Sky', 'correct': False},
    {'answer': 'COEX Tower', 'correct': False}]},
  {'question': 'Which palace is located in the heart of Seoul?',
 

In [28]:
import json

for question in json.loads(response)["questions"]:
    print(question["question"])
    for answer in question["answers"]:
        #print(question["question"])
        
        #print(answer["answer"])
        if answer == {"answer": answer["answer"],"correct": True}:
            print(answer["answer"])

What is the capital city of South Korea?
Seoul
Which river flows through Seoul?
Han River
What is the tallest building in Seoul?
Lotte World Tower
Which palace is located in the heart of Seoul?
Gyeongbokgung Palace
What is the famous shopping district in Seoul?
Myeongdong
